# 06-02-02

- ScalerMinMax
- Eucledian Distance
- Only Transition Matrices
- Remove Constant Features

In [44]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, loadCoverGroups

import numpy as np
from sklearn.preprocessing import StandardScaler
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures, scalerMinMax

In [45]:
cover = loadCoverGroups()
listCovers = cover[12]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [46]:
spotifyFeatures, transitionMatrices, df = loadDataFrames()

In [47]:
ids = transitionMatrices.index.tolist()

In [48]:
transitionMatrices.sum(axis = 1).sum(axis=0)

5979.999999999996

In [49]:
transitionMatrices = removeConstantFeatures(transitionMatrices)

In [50]:
transitionMatrices = scalerMinMax(transitionMatrices)

In [51]:
transitionMatrices.describe()

,0,1,2,3,4,5,6,7,8,9,...,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036
count,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,...,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000
mean,0.028882,0.002190,0.000790,0.000797,0.001207,0.006105,0.022190,0.006024,0.000738,0.001866,...,0.000480,0.000834,0.001752,0.001237,0.000387,0.000167,0.002667,0.000716,0.000167,0.000992
std,0.072392,0.023851,0.018939,0.016458,0.024314,0.037906,0.083196,0.037311,0.019278,0.024825,...,0.018682,0.019811,0.023745,0.023198,0.014235,0.012932,0.026839,0.020779,0.012932,0.022374
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.021205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Applying eucledian distance

In [52]:
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

In [53]:
transitionMatrices

,0,1,2,3,4,5,6,7,8,9,...,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036
0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.219907,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.126509,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.034926,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.016493,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.036458,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5975,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5976,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5977,0.000000,0.043956,0.0,0.073077,0.0,0.035897,0.086610,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5978,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.201058,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.238095,0.0,0.0,0.0,0.547619,0.0,0.0


In [54]:
transitionMatrices['id'] = ids
transitionMatrices = transitionMatrices.set_index('id')

In [55]:
transitionMatrices

,0,1,2,3,4,5,6,7,8,9,...,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036
id,,,,,,,,,,,,,,,,,,,,,
23,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
105,0.219907,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
118,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.126509,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
120,0.034926,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
292,0.016493,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.036458,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002496,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
6002497,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
6002498,0.000000,0.043956,0.0,0.073077,0.0,0.035897,0.086610,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [56]:
originalSongVector = transitionMatrices.loc[[originalSongId]].iloc[0]

In [57]:
ranking = []
for index in range(0, len(transitionMatrices)):
    ranking.append([transitionMatrices.index[index], euclidean_distance(transitionMatrices.iloc[index, :], originalSongVector)])


In [58]:
ranksDF = pd.DataFrame(ranking)

In [59]:
ranksDF = ranksDF.rename(columns={0: "id", 1: "rank"})

In [60]:
ranksDF = ranksDF.sort_values(by="rank")

In [61]:
ranksDF.reset_index(drop=True, inplace=True)

In [62]:
ranksDF

,id,rank
0,6001918,0.000000
1,6000373,0.457261
2,6001599,0.568936
3,6000844,0.663054
4,6002002,0.665455
...,...,...
5975,6002253,3.158344
5976,6001439,3.170182
5977,6002278,3.255601
5978,6002260,3.292349


In [63]:
ranksDF[ranksDF['id']==coverSongId]

,id,rank
4258,6001997,1.538729


In [64]:
ranksDF[ranksDF['id'].isin(listCovers)]

,id,rank
0,6001918,0.000000
4258,6001997,1.538729
